# Coordinate Systems Research

## Setup

In [1]:
!pip3 install geopy nbformat pandas pyproj

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import math
import timeit

import numpy as np
import pandas as pd
from geopy import distance
from pyproj import Geod, Proj
from typing import NamedTuple, Tuple

In [3]:
GEODESIC = Geod(ellps="WGS84")
EARTH_RADIUS_KM = 6378.137

In [4]:
class LatLon(NamedTuple):
    latitude: float
    longitude: float


class XY(NamedTuple):
    x: float
    y: float


def cartesian_to_true_bearing(cartesian: float):
    """Convert a cartesian angle (0 is east, ccw) to true bearing (0 is north, cw).

    Examples
    - 0 -> 90
    - 90 -> 0
    - 180 -> 270
    - 270 -> 180
    """
    return (90 - cartesian + 360) % 360

## Speed of Geographical Distance Implementations

Geographical distance is the distance measured along the surface of the earth.
In Raye we calculate it using the [geopy](https://geopy.readthedocs.io/en/stable/index.html#)
library. The purpose of this experiment is to find the fastest geographical distance implementation.

Implementations that we will be comparing:

- [`geopy.distance.distance`](https://geopy.readthedocs.io/en/stable/index.html#module-geopy.distance)
- [`pyproj.Geod.inv`](https://pyproj4.github.io/pyproj/stable/api/geod.html#pyproj.Geod.inv)

Create wrappers around the implementations so that they can be easily called:

In [5]:
start = LatLon(49.128211, -123.191820)
dest = LatLon(49.148232, -123.158331)


def get_dist_geopy():
    return distance.distance(start, dest).kilometers


def get_dist_pyproj():
    return GEODESIC.inv(start.longitude, start.latitude, dest.longitude, dest.latitude)[2] / 1000

Check that implementations return the same value:

In [6]:
assert math.isclose(get_dist_geopy(), get_dist_pyproj())

Get the average runtime:

In [7]:
geopy_time = timeit.timeit(get_dist_geopy, number=10000)
pyproj_time = timeit.timeit(get_dist_pyproj, number=10000)

f"{geopy_time=:.6f}s, {pyproj_time=:.6f}s"

'geopy_time=1.383742s, pyproj_time=0.010706s'

Conclusion: the pyproj implementation is much faster than geopy. This makes sense, as pyproj is an interface to the C++ library [PROJ](https://proj.org/en/9.2/), whereas geopy is purely in Python.

In [8]:
f"pyproj is {geopy_time / pyproj_time:.2f}x faster than geopy"

'pyproj is 129.25x faster than geopy'

## Accuracy of Projection Methods

To simplify things, our pathfinding algorithms operate on a 2D grid.
However, the position data that we receive is geographical coordinates on the earth, an ellipsoid.
Thus, to solve for a path, we project a section of the earth encompassing the current position and next global waypoint
onto a 2D plane.

There are a number of projection methods that are commonly used. The purpose of this experiment is to
find the best projection method in terms of distance and course accuracy.

Projection methods that we will be comparing:

- `appr`: [Approximate lookup](https://stackoverflow.com/a/1253545)
- `equi`: [Equirectangular projection](https://stackoverflow.com/a/16271669)
- `raye`: [Raye](https://github.com/UBCSailbot/raye-local-pathfinding/blob/7e1d4ce903ac78a41580c76566e84313d11185ba/python/utilities.py#L117-L133)
- [Pyproj projections](https://proj.org/en/9.2/operations/projections/index.html)
- `trap`: Jamen's trapezoid-based method
- `geod`: Using pyproj's `inv` function which return the distance and course given 2 coordinates

In [9]:
def latlon2xy(
    proj: str,
    reference: LatLon,
    latlon: LatLon,
    aspect_ratio: float = None,
    geodesic: Geod = None,
    p: Proj = None,
) -> XY:
    if proj == "appr":
        xy = appr(reference, latlon, aspect_ratio)
    elif proj == "equi":
        xy = equi(reference, latlon, aspect_ratio)
    elif proj == "raye":
        xy = latlon_to_xy(latlon=latlon, reference=reference, geodesic=geodesic)
    elif proj == "trap":
        xy = trap(reference, latlon)
    elif proj == "geod":
        xy = geod(reference, latlon, geodesic)
    else:
        xy = pyproj(proj, reference, latlon, p)
    return XY(*xy)


def appr(reference: LatLon, latlon: LatLon, aspect_ratio: float) -> Tuple[float, float]:
    if aspect_ratio is None:
        aspect_ratio = math.cos(math.radians(np.average((reference.latitude, latlon.latitude))))
    lat_dif = latlon.latitude - reference.latitude
    lon_dif = latlon.longitude - reference.longitude
    lat2km = 110.574
    lon2km = 111.320 * aspect_ratio
    x = lon2km * lon_dif
    y = lat2km * lat_dif
    return x, y


def equi(reference: LatLon, latlon: LatLon, aspect_ratio: float) -> Tuple[float, float]:
    if aspect_ratio is None:
        aspect_ratio = math.cos(math.radians(np.average((reference.latitude, latlon.latitude))))
    lat_dif = latlon.latitude - reference.latitude
    lon_dif = latlon.longitude - reference.longitude
    x = EARTH_RADIUS_KM * math.radians(lon_dif) * aspect_ratio
    y = EARTH_RADIUS_KM * math.radians(lat_dif)
    return x, y


def latlon_to_xy(latlon: LatLon, reference: LatLon, geodesic: Geod) -> Tuple[float, float]:
    def get_dist_n_course(start: LatLon, dest: LatLon, geodesic: Geod) -> Tuple[float, float]:
        if geodesic is None:
            geodesic = Geod(ellps="WGS84")
        fwd_azimuth, _, distance = geodesic.inv(
            lons1=start.longitude, lats1=start.latitude, lons2=dest.longitude, lats2=dest.latitude
        )
        return distance / 1000, fwd_azimuth % 360

    # raye method of converting to 2D cartesian
    x, _ = get_dist_n_course(reference, LatLon(reference.latitude, latlon.longitude), geodesic)
    y, _ = get_dist_n_course(reference, LatLon(latlon.latitude, reference.longitude), geodesic)
    if reference.longitude > latlon.longitude:
        x = -x
    if reference.latitude > latlon.latitude:
        y = -y
    return x, y


def trap(reference: LatLon, latlon: LatLon) -> Tuple[float, float]:
    km_per_degree = 2 * math.pi * EARTH_RADIUS_KM / 360.0

    w_side_len = km_per_degree * (latlon.latitude - reference.latitude)
    n_side_len = (
        km_per_degree
        * math.cos(math.radians(max(latlon.latitude, reference.latitude)))
        * (latlon.longitude - reference.longitude)
    )
    s_side_len = (
        km_per_degree
        * math.cos(math.radians(min(latlon.latitude, reference.latitude)))
        * (latlon.longitude - reference.longitude)
    )

    w_side_plus_minus = 1 if (w_side_len > 0) else (-1)

    distance = math.sqrt((n_side_len**2 + s_side_len**2) / 2 + w_side_len**2)
    theta = math.atan2(
        w_side_plus_minus * math.sqrt(w_side_len**2 + ((s_side_len - n_side_len) / 2) ** 2),
        (n_side_len + s_side_len) / 2,
    )

    return (distance * math.cos(theta)), (distance * math.sin(theta))


def geod(reference: LatLon, latlon: LatLon, geodesic: Geod) -> Tuple[float, float]:
    if geodesic is None:
        geodesic = Geod(ellps="WGS84")
    m_to_km = 0.001

    azi12, _, distance = geodesic.inv(
        reference.longitude, reference.latitude, latlon.longitude, latlon.latitude
    )
    distance = distance * m_to_km

    return distance * math.sin(math.radians(azi12)), distance * math.cos(math.radians(azi12))


def pyproj(proj: str, reference: LatLon, latlon: LatLon, p: Proj) -> Tuple[float, float]:
    if p is None:
        p = Proj(
            proj=proj,
            lat_0=reference.latitude,
            lon_0=reference.longitude,
            ellps="WGS84",
            units="km",
        )
    start_xy = p(reference.longitude, reference.latitude)
    dest_xy = p(latlon.longitude, latlon.latitude)
    return np.subtract(dest_xy, start_xy)

Error functions:
- `error` is the typical percentage error equation, but in cases where it would divide by zero
  it returns the absolute difference
- `degree_error` returns the percentage error for circular values with a range of 360

In [10]:
def error(expected, observed):
    diff = abs(expected - observed)
    if expected == 0:
        return diff
    return diff / expected * 100


def degree_error(expected, observed):
    circular_diff = min(abs(expected - observed), 360 - abs(expected - observed))
    error = circular_diff / 360 * 100
    return error

Use each projection method in a variety of different cases, varying:
- Starting latitude
    - We don't expect to be going above 60 degrees
- Course: direction to destination
- Distance: distance to destination
    - We don't expect local pathfinding to navigate to waypoints greater than 20km away

For each method, record:
- Course error
- Distance error
- Time: execution time
- Cached time: execution time when the most expensive part of the method is precomputed and passed in as an argument

In [11]:
df = []

dist_wins = np.zeros(4)
course_wins = np.zeros(4)

start_lon = -123
for start_lat in [0, 15, 30, 45, 60]:
    start = LatLon(latitude=start_lat, longitude=start_lon)
    for course_deg in [0, 45, 90, 210]:
        for dist_km in [1, 2, 5, 10, 20]:
            dest_lon, dest_lat, _ = GEODESIC.fwd(
                lons=start_lon, lats=start_lat, az=course_deg, dist=dist_km * 1000
            )
            dest = LatLon(latitude=dest_lat, longitude=dest_lon)

            for method in ["appr", "equi", "raye", "trap", "geod"]:
                x, y = latlon2xy(proj=method, reference=start, latlon=dest)
                course_proj = cartesian_to_true_bearing(np.rad2deg(np.arctan2(y, x)))
                course_err = degree_error(expected=course_deg, observed=course_proj)
                dist_proj = np.hypot(x, y)
                dist_err = error(expected=dist_km, observed=dist_proj)
                time_proj = timeit.timeit(
                    lambda: latlon2xy(proj=method, reference=start, latlon=dest),
                    number=100,
                )
                if method in ["appr", "equi"]:
                    aspect_ratio = np.cos(np.deg2rad(np.average((start.latitude, dest.latitude))))
                    time_proj_cached = timeit.timeit(
                        lambda: latlon2xy(
                            proj=method, reference=start, latlon=dest, aspect_ratio=aspect_ratio
                        ),
                        number=100,
                    )
                elif method in ["raye", "geod"]:
                    time_proj_cached = timeit.timeit(
                        lambda: latlon2xy(
                            proj=method, reference=start, latlon=dest, geodesic=GEODESIC
                        ),
                        number=100,
                    )
                elif method == "trap":
                    time_proj_cached = time_proj  # not cached
                else:
                    p = Proj(
                        proj=method,
                        lat_0=start.latitude,
                        lon_0=start.longitude,
                        ellps="WGS84",
                        units="km",
                    )
                    time_proj_cached = timeit.timeit(
                        lambda: latlon2xy(proj=method, reference=start, latlon=dest, p=p),
                        number=100,
                    )

                df.append(
                    {
                        "start_lat": start_lat,
                        "course_deg": course_deg,
                        "dist_km": dist_km,
                        "method": method,
                        "course (deg)": course_proj,
                        "course_err (%)": course_err,
                        "dist (km)": dist_proj,
                        "dist_err (%)": dist_err,
                        "time (s)": time_proj,
                        "cached time (s)": time_proj_cached,
                    },
                ),

df = pd.DataFrame(df)
df.head(12)

,start_lat,course_deg,dist_km,method,course (deg),course_err (%),dist (km),dist_err (%),time (s),cached time (s)
0,0,0,1,appr,0.0,0.0,0.999998,2.494530e-04,0.000531,0.000088
1,0,0,1,equi,0.0,0.0,1.006739,6.739497e-01,0.000539,0.000104
2,0,0,1,raye,0.0,0.0,1.000000,2.220446e-14,0.003179,0.000640
3,0,0,1,trap,0.0,0.0,1.006739,6.739497e-01,0.000237,0.000237
4,0,0,1,geod,0.0,0.0,1.000000,2.220446e-14,0.000466,0.000175
5,0,0,2,appr,0.0,0.0,1.999995,2.494780e-04,0.000501,0.000103
6,0,0,2,equi,0.0,0.0,2.013479,6.739496e-01,0.000508,0.000097
7,0,0,2,raye,0.0,0.0,2.000000,2.220446e-14,0.000959,0.000445
8,0,0,2,trap,0.0,0.0,2.013479,6.739496e-01,0.000217,0.000217
9,0,0,2,geod,0.0,0.0,2.000000,2.220446e-14,0.000455,0.000172


Cases with the worst course error

In [12]:
df[df["course_err (%)"] > 0.05]

,start_lat,course_deg,dist_km,method,course (deg),course_err (%),dist (km),dist_err (%),time (s),cached time (s)
26,0,45,1,equi,44.807577,0.053451,1.003375,0.337541,0.000482,0.000093
28,0,45,1,trap,44.807577,0.053451,1.003375,0.337541,0.000211,0.000211
31,0,45,2,equi,44.807577,0.053451,2.006751,0.337541,0.000479,0.000094
33,0,45,2,trap,44.807577,0.053451,2.006751,0.337540,0.000212,0.000212
36,0,45,5,equi,44.807580,0.053450,5.016877,0.337541,0.000484,0.000093
38,0,45,5,trap,44.807579,0.053450,5.016877,0.337539,0.000212,0.000212
41,0,45,10,equi,44.807590,0.053447,10.033754,0.337543,0.000547,0.000100
43,0,45,10,trap,44.807586,0.053448,10.033754,0.337535,0.000225,0.000225
46,0,45,20,equi,44.807630,0.053436,20.067510,0.337549,0.000501,0.000096
48,0,45,20,trap,44.807612,0.053441,20.067504,0.337518,0.000219,0.000219


Cases with the worst distance error

In [13]:
df[df["dist_err (%)"] > 0.5]

,start_lat,course_deg,dist_km,method,course (deg),course_err (%),dist (km),dist_err (%),time (s),cached time (s)
1,0,0,1,equi,0.000000,0.000000,1.006739,0.673950,0.000539,0.000104
3,0,0,1,trap,0.000000,0.000000,1.006739,0.673950,0.000237,0.000237
6,0,0,2,equi,0.000000,0.000000,2.013479,0.673950,0.000508,0.000097
8,0,0,2,trap,0.000000,0.000000,2.013479,0.673950,0.000217,0.000217
11,0,0,5,equi,0.000000,0.000000,5.033697,0.673949,0.000536,0.000100
13,0,0,5,trap,0.000000,0.000000,5.033697,0.673949,0.000214,0.000214
16,0,0,10,equi,0.000000,0.000000,10.067395,0.673949,0.000503,0.000098
18,0,0,10,trap,0.000000,0.000000,10.067395,0.673949,0.000209,0.000209
21,0,0,20,equi,0.000000,0.000000,20.134789,0.673946,0.000491,0.000095
23,0,0,20,trap,0.000000,0.000000,20.134789,0.673946,0.000208,0.000208


Mean execution time for each method

In [14]:
df.groupby(["method"]).mean().iloc[:, -2:]

,time (s),cached time (s)
method,,
appr,0.000497,0.000088
equi,0.000507,0.000096
geod,0.000460,0.000196
raye,0.001011,0.000429
trap,0.000223,0.000223


Conclusion: `geod` offers the best accuracy (uses accurate model of the Earth) and is 2x faster than Raye's method.